In [1]:
from decouple import config
from openai import OpenAI

In [2]:
OPENAI_API_KEY = config("OPENAI_API_KEY")
UPSTASH_VECTOR_REST_URL = config("UPSTASH_VECTOR_REST_URL")
UPSTASH_VECTOR_REST_TOKEN = config("UPSTASH_VECTOR_REST_TOKEN")

In [3]:
client = OpenAI(api_key=OPENAI_API_KEY)

In [4]:
def get_embedding(text, model="text-embedding-3-small"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

In [9]:
documents = [
    "The cat jumped over the dog",
    "The cow jumped over the moon",
    "The turkey ran in circles",
]


In [10]:

embeddings = [get_embedding(x) for x in documents]

In [11]:
dataset = {}
for i, embedding in enumerate(embeddings):
    dataset[i] = embedding


In [12]:
from upstash_vector import Index, Vector

index = Index(url=UPSTASH_VECTOR_REST_URL, token=UPSTASH_VECTOR_REST_TOKEN)


In [14]:
vectors=[]
for key,value in dataset.items():
    print(key)
    my_id=key
    embedding=value
    vectors.append(Vector(id=my_id,vector=embedding))

0
1
2


In [17]:
index.upsert(vectors=vectors)

'Success'

In [23]:
dataset[2]=get_embedding("The moose sat near the dog")

In [25]:
index.upsert(vectors=[Vector(id=2,vector=dataset[2])])

'Success'

In [26]:
query_string="The moose sat over the dog"
query_embedding=get_embedding(query_string)

In [27]:
results=index.query(
  vector=query_embedding,
  top_k=3,
  include_vectors=True,
  include_metadata=True,
)
for result in results:
    print(result.id,result.score)

2 0.9586149
0 0.7527521
1 0.7056093
